> ##### 示例 2：过程导向的策略
>
> ![Policy-based](https://gibberblot.github.io/rl-notes/_images/6570a33405c74bc15cf593e8dd9ea9909aa686c3d7e3aeafe5f00dfd0453e1a1.png)
> 
> 这个示例描述了一个更复杂的**马尔可夫决策过程 (Markov Decision Process - MDP)** 任务，通常被称为“对抗性穿越”（Contested Crossing）。它比上一个网格世界的例子复杂得多，因为它引入了：
> 
> 1.  **更复杂的环境：** 六边形网格，不同类型的地形。
> 2.  **多目标和风险：** 不仅仅是到达目的地，还要考虑敌人的攻击和自身的健康状况。
> 3.  **不确定性：** 行动结果、敌人攻击、甚至船的移动都存在不确定性。
> 4.  **状态空间更大：** 智能体不仅仅需要考虑自己的位置，还需要考虑敌人的位置、敌人的健康状况、自己的健康状况等。
> 
> 我们来详细分解一下：
> 
> **1. 智能体 (Agent)：**
> 
> * **智能体类型：** 一艘船。
> * **初始位置：** 在水体的南岸（图中的蓝色小船）。
> * **移动方式：** 船可以在六边形网格的水域（浅灰色）中航行，但不能在陆地（深灰色）上。
> * **行动：** 每一步都可以选择一个方向（西、西北、东北、东、东南或西南）移动。
> * **目标：** 到达北岸。北岸最近的点用星形字符表示。
> 
> **2. 敌人 (Enemy)：**
> 
> * **敌人位置：** 在北岸，由黄色或红色星星表示的区域。
> * **敌人行为：**
>     * 当船到达敌人的危险区域（黄色或红色星星区域）时，敌人会向船射击。
>     * 敌人的射击是**完全确定**的，并且不需要选择（一旦船进入危险区域，敌人就会开火）。
> 
> **3. 奖励 (Rewards)：**
> 
> * **到达北岸：** 奖励 +10。
> * **沉没 (Sinking)：** 惩罚 -10（负值）。
> 
> **4. 复杂性与不确定性 (Critical Part!)：**
> 
> 这里是这个MDP最复杂的部分，它引入了大量的状态和转移不确定性。
> 
> * **船的射击能力：**
>     * 在有黄色或红色星星的区域，船也可以向敌人射击。
>     * 如果船选择射击，它**不能同时移动**，而是会继续留在原地。
>     * **每次射击有 10% 的概率对目标造成伤害。** (注意，这里只提了10%伤害目标，没有明确说是否射击失败就不造成伤害，或者有其他副作用。)
> 
> * **船的受损情况 (Damage Levels)：**
>     * **在低危险区（黄色星星区域）：** 敌人射击时，有 **10% 的概率**会使船受损（无论是船自身射击敌人还是敌人射击船）。
>     * **在高危险区（红色星星区域）：** 敌人射击时，有 **99% 的概率**会使船受损。
>     * **船的移动成功率与损伤等级有关：**
>         * **满血 (Full health)：** 船 **100%** 成功移动。
>         * **损伤等级 1：** 船 **67%** 成功移动，**33%** 会沉没。
>         * **损伤等级 2：** 船 **33%** 成功移动，**67%** 会沉没。
>         * **损伤等级 3：** 船 **100%** 沉没。
>         * （注意：描述中没有明确说明损伤等级如何增加，通常是通过受到伤害。但从“受到伤害”的概率看，这里损伤等级可能指的是生命值从高到低的分类）
> 
> * **敌人受损情况 (Enemy Damage Levels)：**
>     * 当敌人受到损伤等级 1 时，它的行为**不变**。
>     * 当敌人受到损伤等级 2 时，它**被摧毁**，不再构成威胁。此时船就没有进一步的威胁了。
> 
> * **船的感知能力：**
>     * 船可以观察到**整个环境的状态**：
>         * 它自己的位置。
>         * 敌人的位置。
>         * 它自己的健康状况（损伤等级）。
>         * 敌人的健康状况（损伤等级）。
> 
> **5. 任务目标：**
> 
> * **导航到高奖励区：** 船需要从起始位置导航到一个能获得奖励的地方。
> * **复杂性：**
>     * 决定最佳行动方案非常复杂。
>     * 存在**多种不同**的高奖励结束状态（可能指到达不同的北岸点，或者在不同健康状况下到达）。
>     * 有奖励很慢但能保证实现路径，也有更快但风险更高的路径。这意味着智能体需要权衡**风险与回报**。